### 1.d. - Autitoria no nome das ruas (Street)

Para identificação de objetos do tipo Ruas, Avenidas, Rodovias, etc., utilizamos a seguinte regra:

- Tags "way" e subtag tag atributo "name"
<br/>Ou<br;> 
- tag "node" e subtag "tag" e atributo "add:street"

No entanto, para o primeiro caso, existem uma infinidade de elementos que não são ruas. O código abaixo realiza alguns tratamentos na tentativa de identificar tipos que são ruas.

Foram levantados as identificações mais comuns (variável expected), como Rua, Avenida, Rodovia, etc. O que estiver fora destes padrões, serão considerados inconsistentes e deverão ser analisados.

Os nomes de ruas com caracteres especiais também serão considerados como inconsistêntes.

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "../dados/jundiai_e_regiao_map_zen.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

accept = re.compile(r'^\w*$')
upper = re.compile(r'^([A-Z])*$')
lower_and_upper = re.compile(r'^([A-z]| )*$')
problemchars = re.compile(r'[=\+/&<>;"\?%#$@\,\t\r\n]')
with_number = re.compile(r'^[0-9]')

# O que for diferente destes tipos, devem obrigatoriamente ser exibidos
expected = ["Rua", "Avenida", "Rodovia", "Travessa", "Viaduto", "Alameda", "Praça", "Via", "Estrada",
            "Viela", "Calçadão", "Largo", "Rodoanel", "Rotatória", "Alameda", "Caminho", "Corredor",
            "Passagem", "Acesso", "Marginal", "Trevo", "Passarela", "Contorno", "Travessia",
            "Paço", "Entrada", "Túnel", "Entrada", "Travessia", "Saída", "Alamedas", "Vila", "Trilha"]

def audit_street_type(street_types, street_name):
    result = False
    
    street_type = street_name.split()[0]
    if street_type not in expected:
        street_types[street_type].add(street_type)
        result = False
    else:
        result = True

    return result


def audit_characters_street(element):
    result = False
    classificated = False
    
    # UPPER
    if upper.search(element) is not None:
        classificated = True
    
    # LOWER
    if not classificated:    
        if accept.search(element) is not None:
            result = True
            classificated = True
            classificated = True        
    
    # LOWER AND UPPER
    if not classificated:
        if lower_and_upper.search(element) is not None:
            result = True
            classificated = True        
            
    # with_number
    if not classificated:
        if with_number.search(element) is not None:        
            result = True
            classificated = True        

    # PROBLEMCHAR
    if problemchars.search(element) is not None:
        classificated = True        
        
    # OUTROS
    if not classificated:
        result = True
    
    return result


def audit(osmfile):
    osm_file = open(osmfile, "r", encoding="utf-8")
    street_types = defaultdict(set)
    id_tag_highway = []
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
                
        if elem.tag == "node" or elem.tag == "way":
     
            # Se a tag for WAY, deve-ser haver a subtag k = HIGHWAY para ser considerado uma Rua 
            if elem.tag == "way":
                for tag in elem.iter("tag"):
                    if tag.attrib['k'] == "highway":
                        if elem.attrib["id"] not in id_tag_highway:
                            id_tag_highway.append(elem.attrib["id"])
                        break
            
            # Se não estiver entre os tipos de ruas esperados, imprimir. Caso contrário, validar os caracteres
            for tag in elem.iter("tag"):
                if (elem.tag == "way" and tag.attrib['k'] == "name"):
                    if elem.attrib["id"] in id_tag_highway:
                        if audit_street_type(street_types, tag.attrib['v']):
                            if audit_characters_street(tag.attrib['v']) == False:
                                print(tag.attrib)                                
                        else:
                            print(tag.attrib)

                elif (elem.tag == "node" and tag.attrib['k'] == "addr:street"):
                    if audit_street_type(street_types, tag.attrib['v']):
                        if audit_characters_street(tag.attrib['v']) == False:
                            print(tag.attrib)
                    else:
                        print(tag.attrib)
                
    osm_file.close()
    return street_types


def test():
    # Auditar para encontrar nome das ruas fora do padrão
    st_types = audit(OSMFILE)      
            

if __name__ == '__main__':
    test()


{'k': 'addr:street', 'v': 'estrada Tahira Eki'}
{'k': 'name', 'v': 'Rua " 30 "'}
{'k': 'name', 'v': 'Rua " 26 "'}
{'k': 'name', 'v': 'Rua "1"'}
{'k': 'name', 'v': 'Rua "2"'}
{'k': 'name', 'v': 'Complexo Viário Tobias Muzaiel'}
{'k': 'name', 'v': 'Rua "04"'}
{'k': 'name', 'v': 'Avenida dos Alpes; Avenida Pauliceia'}
{'k': 'name', 'v': 'Avenidas das Aves Marinhas'}
{'k': 'name', 'v': 'Rau Diogo Alveres correia'}
{'k': 'name', 'v': 'rua Raul Breesane Malta'}
{'k': 'name', 'v': 'Miguel De Barros'}
{'k': 'name', 'v': 'José Pereira da Silva'}
{'k': 'name', 'v': 'Rod. Manoel Silvério Pinto'}
{'k': 'name', 'v': 'Rua Vitoria,'}
{'k': 'name', 'v': 'Rua "R-5"'}
{'k': 'name', 'v': 'José Vitórino Ferreira Filho'}
{'k': 'name', 'v': 'Rua Manoel Pinto Rodrigues,'}
{'k': 'name', 'v': 'Rua " 16 "'}
{'k': 'name', 'v': 'Pedro Barsanelli'}
{'k': 'name', 'v': 'Antiga Estrada da Mata Fria'}
{'k': 'name', 'v': 'Ponte Velha'}
{'k': 'name', 'v': 'Doutor Osvaldo Urioste'}
{'k': 'name', 'v': 'Ru Fernão Dias'}
{'